# 01. データ準備 - S3画像収集 & Ground Truthアノテーションジョブ作成

## 概要
1. S3に蓄積されたメーター画像を一覧取得
2. メーター種別（hygrometer / pressure）ごとにフィルタリング
3. Ground Truth用のマニフェストファイルを作成
4. カスタムUIテンプレート（crowd-keypoint）を作成
5. Ground Truthアノテーションジョブを起動

## 1. セットアップ

In [ ]:
import boto3
import json
import os
from datetime import datetime
from pathlib import Path

# AWS クライアント
s3 = boto3.client('s3')
sagemaker_client = boto3.client('sagemaker')
dynamodb = boto3.resource('dynamodb')

# 設定
SOURCE_BUCKET = 'facteye-images-20251114'  # 既存の画像バケット
SAGEMAKER_BUCKET = None  # 後で SageMaker デフォルトバケットを取得
REGION = 'us-east-1'
PREFIX = 'images/'  # S3上の画像プレフィックス

# SageMaker デフォルトバケット取得
import sagemaker
sess = sagemaker.Session()
SAGEMAKER_BUCKET = sess.default_bucket()
ROLE = sagemaker.get_execution_role()

print(f'Source Bucket: {SOURCE_BUCKET}')
print(f'SageMaker Bucket: {SAGEMAKER_BUCKET}')
print(f'Role: {ROLE}')

## 2. S3から画像一覧を取得

In [ ]:
def list_s3_images(bucket, prefix='images/', suffix='.jpg'):
    """S3バケットから画像ファイルの一覧を取得する"""
    images = []
    paginator = s3.get_paginator('list_objects_v2')
    
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get('Contents', []):
            key = obj['Key']
            if key.endswith(suffix):
                images.append({
                    'key': key,
                    'size': obj['Size'],
                    'last_modified': obj['LastModified']
                })
    
    return images

all_images = list_s3_images(SOURCE_BUCKET)
print(f'総画像数: {len(all_images)}')

## 3. Metersテーブルからメーター種別情報を取得

In [ ]:
def get_meter_device_mapping():
    """Metersテーブルからdevice_id → meter_type のマッピングを取得"""
    table = dynamodb.Table('Meters')
    mapping = {}  # device_id -> list of meter info
    
    response = table.scan(
        FilterExpression='attribute_not_exists(deleted_at)'
    )
    
    for item in response.get('Items', []):
        device_id = item.get('device_id')
        if device_id:
            if device_id not in mapping:
                mapping[device_id] = []
            mapping[device_id].append({
                'meter_id': item['meter_id'],
                'meter_type': item.get('meter_type', 'unknown'),
                'company_id': item['company_id']
            })
    
    # ページネーション対応
    while 'LastEvaluatedKey' in response:
        response = table.scan(
            FilterExpression='attribute_not_exists(deleted_at)',
            ExclusiveStartKey=response['LastEvaluatedKey']
        )
        for item in response.get('Items', []):
            device_id = item.get('device_id')
            if device_id:
                if device_id not in mapping:
                    mapping[device_id] = []
                mapping[device_id].append({
                    'meter_id': item['meter_id'],
                    'meter_type': item.get('meter_type', 'unknown'),
                    'company_id': item['company_id']
                })
    
    return mapping

meter_mapping = get_meter_device_mapping()
print(f'デバイス数: {len(meter_mapping)}')
for device_id, meters in meter_mapping.items():
    types = [m['meter_type'] for m in meters]
    print(f'  {device_id}: {types}')

## 4. メーター種別でフィルタリング

S3パスの `device_id` とMetersテーブルの `meter_type` を突き合わせて、対象のメーター種別の画像のみ抽出する。

In [ ]:
# 対象メーター種別
TARGET_METER_TYPES = ['hygrometer', 'pressure']

def parse_s3_key(key):
    """S3キーからcompany_id, device_idを抽出する
    
    パス形式: images/{company_id}/{device_id}/YYYY/MM/DD/HHMMSS.jpg
    """
    parts = key.split('/')
    if len(parts) >= 3:
        return {
            'company_id': parts[1],
            'device_id': parts[2]
        }
    return None

def filter_images_by_meter_type(images, meter_mapping, target_types):
    """メーター種別で画像をフィルタリング"""
    filtered = {t: [] for t in target_types}
    
    for img in images:
        parsed = parse_s3_key(img['key'])
        if not parsed:
            continue
        
        device_id = parsed['device_id']
        if device_id in meter_mapping:
            for meter in meter_mapping[device_id]:
                if meter['meter_type'] in target_types:
                    filtered[meter['meter_type']].append({
                        **img,
                        **parsed,
                        'meter_id': meter['meter_id'],
                        'meter_type': meter['meter_type']
                    })
    
    return filtered

filtered_images = filter_images_by_meter_type(all_images, meter_mapping, TARGET_METER_TYPES)

for meter_type, imgs in filtered_images.items():
    print(f'{meter_type}: {len(imgs)}枚')

## 5. Ground Truth マニフェストファイル作成

Ground Truthジョブに必要な入力マニフェスト（JSONL形式）を生成する。

In [ ]:
def create_manifest(images, bucket, output_path):
    """Ground Truth用のマニフェストファイルを作成
    
    各行のJSON形式:
    {"source-ref": "s3://bucket/key", "meter_type": "...", "meter_id": "..."}
    """
    manifest_lines = []
    
    for img in images:
        entry = {
            'source-ref': f's3://{bucket}/{img["key"]}',
            'meter_type': img.get('meter_type', 'unknown'),
            'meter_id': img.get('meter_id', 'unknown'),
            'device_id': img.get('device_id', 'unknown'),
            'company_id': img.get('company_id', 'unknown')
        }
        manifest_lines.append(json.dumps(entry))
    
    manifest_content = '\n'.join(manifest_lines)
    
    # S3にアップロード
    s3.put_object(
        Bucket=SAGEMAKER_BUCKET,
        Key=output_path,
        Body=manifest_content.encode('utf-8'),
        ContentType='application/jsonl'
    )
    
    print(f'マニフェスト作成完了: s3://{SAGEMAKER_BUCKET}/{output_path}')
    print(f'  画像数: {len(manifest_lines)}')
    return f's3://{SAGEMAKER_BUCKET}/{output_path}'

# 全対象画像をまとめたマニフェストを作成
all_target_images = []
for meter_type, imgs in filtered_images.items():
    all_target_images.extend(imgs)

timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
manifest_s3_uri = create_manifest(
    all_target_images,
    SOURCE_BUCKET,
    f'ground-truth/manifests/input-{timestamp}.manifest'
)

print(f'\n合計画像数: {len(all_target_images)}')

## 6. カスタムUIテンプレート（crowd-keypoint）

アナログメーターのキーポイントアノテーション用UIテンプレートを作成する。

### キーポイント定義
| キーポイント | 説明 |
|-------------|------|
| `needle_tip` | 針の先端 |
| `needle_center` | 針の回転中心（軸） |
| `scale_min` | スケール最小値の位置 |
| `scale_max` | スケール最大値の位置 |

In [ ]:
UI_TEMPLATE = """
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
  <crowd-keypoint
    src="{{ task.input.source-ref | grant_read_access }}"
    labels="['needle_tip', 'needle_center', 'scale_min', 'scale_max']"
    header="メーターのキーポイントをアノテーションしてください"
    name="keypoints"
  >
    <full-instructions header="アノテーション手順">
      <h2>キーポイントの定義</h2>
      <p>各メーター画像に対して、以下の4つのキーポイントを正確にマークしてください。</p>
      <table border="1" cellpadding="5">
        <tr><th>キーポイント</th><th>説明</th><th>注意事項</th></tr>
        <tr>
          <td><strong>needle_tip</strong></td>
          <td>針の先端</td>
          <td>針が指し示している最も外側の端をマーク</td>
        </tr>
        <tr>
          <td><strong>needle_center</strong></td>
          <td>針の回転中心（軸）</td>
          <td>針が回転する中心点をマーク</td>
        </tr>
        <tr>
          <td><strong>scale_min</strong></td>
          <td>スケール最小値の位置</td>
          <td>目盛りの最小値（0やMINなど）の位置をマーク</td>
        </tr>
        <tr>
          <td><strong>scale_max</strong></td>
          <td>スケール最大値の位置</td>
          <td>目盛りの最大値の位置をマーク</td>
        </tr>
      </table>

      <h2>画像がぼやけている場合</h2>
      <p>画像がぼやけていて判別困難な場合でも、最善の推測でキーポイントをマークしてください。</p>
      <p>どうしても判別できない場合は、全てのキーポイントを画像の中心に配置してください（後処理で除外します）。</p>

      <h2>メーター種別: {{ task.input.meter_type }}</h2>
    </full-instructions>

    <short-instructions>
      <p>4つのキーポイントをマークしてください:</p>
      <ol>
        <li><strong>needle_tip</strong>: 針の先端</li>
        <li><strong>needle_center</strong>: 針の回転中心</li>
        <li><strong>scale_min</strong>: スケール最小値</li>
        <li><strong>scale_max</strong>: スケール最大値</li>
      </ol>
      <p>メーター種別: {{ task.input.meter_type }}</p>
    </short-instructions>
  </crowd-keypoint>
</crowd-form>
"""

# UIテンプレートをS3にアップロード
template_s3_key = 'ground-truth/ui-template/keypoint-template.liquid.html'
s3.put_object(
    Bucket=SAGEMAKER_BUCKET,
    Key=template_s3_key,
    Body=UI_TEMPLATE.encode('utf-8'),
    ContentType='text/html'
)

print(f'UIテンプレートアップロード完了: s3://{SAGEMAKER_BUCKET}/{template_s3_key}')

## 7. プライベートワークフォース確認

既存のプライベートワークフォースを確認する。存在しない場合はSageMakerコンソールから作成する。

In [ ]:
# プライベートワークフォースの確認
try:
    workforces = sagemaker_client.list_workforces()
    if workforces['Workforces']:
        workforce = workforces['Workforces'][0]
        workforce_arn = workforce['WorkforceArn']
        print(f'ワークフォース検出: {workforce["WorkforceName"]}')
        print(f'ARN: {workforce_arn}')
    else:
        print('ワークフォースが見つかりません。')
        print('SageMakerコンソール > Ground Truth > Labeling workforces から作成してください。')
except Exception as e:
    print(f'ワークフォース確認エラー: {e}')
    print('SageMakerコンソールからプライベートワークフォースを作成してください。')

In [ ]:
# プライベートワークチームの確認
try:
    workteams = sagemaker_client.list_workteams()
    if workteams['Workteams']:
        for team in workteams['Workteams']:
            print(f'ワークチーム: {team["WorkteamName"]}')
            print(f'  ARN: {team["WorkteamArn"]}')
    else:
        print('ワークチームが見つかりません。')
        print('SageMakerコンソールからワークチームを作成してください。')
except Exception as e:
    print(f'ワークチーム確認エラー: {e}')

## 8. Ground Truth アノテーションジョブ作成

**前提条件:**
- プライベートワークフォース & ワークチームが作成済みであること
- 上記セルで `WORKTEAM_ARN` を確認済みであること

In [ ]:
# === ジョブ設定 ===
# ワークチームARNを設定（上のセルで確認した値を入力）
WORKTEAM_ARN = 'arn:aws:sagemaker:us-east-1:<ACCOUNT_ID>:workteam/private-crowd/<TEAM_NAME>'  # TODO: 実際のARNに置換

JOB_NAME = f'facteye-meter-keypoint-{timestamp}'
OUTPUT_S3_URI = f's3://{SAGEMAKER_BUCKET}/ground-truth/output/'

print(f'ジョブ名: {JOB_NAME}')
print(f'入力マニフェスト: {manifest_s3_uri}')
print(f'出力先: {OUTPUT_S3_URI}')
print(f'ワークチーム: {WORKTEAM_ARN}')

In [ ]:
# ラベリングジョブの作成
labeling_job_config = {
    'LabelingJobName': JOB_NAME,
    'LabelAttributeName': 'keypoints',
    'InputConfig': {
        'DataSource': {
            'S3DataSource': {
                'ManifestS3Uri': manifest_s3_uri
            }
        }
    },
    'OutputConfig': {
        'S3OutputPath': OUTPUT_S3_URI
    },
    'RoleArn': ROLE,
    'HumanTaskConfig': {
        'WorkteamArn': WORKTEAM_ARN,
        'UiConfig': {
            'UiTemplateS3Uri': f's3://{SAGEMAKER_BUCKET}/{template_s3_key}'
        },
        'PreHumanTaskLambdaArn': f'arn:aws:lambda:{REGION}:081040173940:function:PRE-Keypoint',
        'TaskTitle': 'FactEye メーターキーポイントアノテーション',
        'TaskDescription': 'アナログメーター画像にキーポイント（針先端・中心・最小値・最大値）をマークしてください',
        'NumberOfHumanWorkersPerDataObject': 1,
        'TaskTimeLimitInSeconds': 300,
        'AnnotationConsolidationConfig': {
            'AnnotationConsolidationLambdaArn': f'arn:aws:lambda:{REGION}:081040173940:function:ACS-Keypoint'
        }
    }
}

print('ジョブ設定:')
print(json.dumps(labeling_job_config, indent=2, default=str))

In [ ]:
# ジョブ起動（実行前に設定を確認してください）
# response = sagemaker_client.create_labeling_job(**labeling_job_config)
# print(f'ラベリングジョブ起動完了: {JOB_NAME}')
# print(f'ARN: {response["LabelingJobArn"]}')

print('=== ジョブ起動はコメントアウトされています ===')
print('設定を確認後、上のコメントを解除して実行してください。')
print(f'  - WORKTEAM_ARN が正しいか確認')
print(f'  - マニフェストの画像数: {len(all_target_images)}')

## 9. ジョブの進捗確認

In [ ]:
def check_labeling_job_status(job_name):
    """ラベリングジョブの進捗を確認"""
    response = sagemaker_client.describe_labeling_job(LabelingJobName=job_name)
    
    status = response['LabelingJobStatus']
    counters = response.get('LabelCounters', {})
    
    print(f'ジョブ名: {job_name}')
    print(f'ステータス: {status}')
    print(f'総データ数: {counters.get("TotalLabeled", 0) + counters.get("Unlabeled", 0)}')
    print(f'ラベル済み: {counters.get("TotalLabeled", 0)}')
    print(f'未ラベル: {counters.get("Unlabeled", 0)}')
    print(f'失敗: {counters.get("FailedNonRetryableError", 0)}')
    
    if status == 'Completed':
        output_uri = response['OutputConfig']['S3OutputPath']
        print(f'\n出力先: {output_uri}')
        print(f'出力マニフェスト: {response.get("LabelingJobOutput", {}).get("OutputDatasetS3Uri", "N/A")}')
    
    return status

# ジョブ名を指定して確認
# check_labeling_job_status(JOB_NAME)

## 次のステップ

Ground Truthジョブが完了したら、**02_annotation_to_yolo.ipynb** でアノテーション結果をYOLOv8-pose形式に変換します。